# Das Diet-Problem

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/opt2025-de/blob/main/LP-Diet.ipynb)


## Problemstellung

Das Ziel des Diet-Problems ist es, Mengen von Lebensmitteln zu finden, die die täglichen Ernährungsbedürfnisse eines Menschen erfüllen, wobei die Kosten minimiert werden. Der Datensatz
`dt` enthält die Nährstoffinformationen und die Kosten für 6 Lebensmittel. In diesem kleinen Beispiel werden wir die täglichen Ernährungsbedürfnisse eines Menschen in Bezug auf die Nährstoffe Protein, Calcium, Eisen und Vitamin C betrachten.

Eine Umsetzung in Excel finden Sie [hier](https://github.com/febse/data/raw/refs/heads/main/opt/diet.xlsx).

In [36]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Create the DataFrame
df = pd.DataFrame( {
    "Food": ["Oatmeals", "Chicken", "Eggs", "Milk", "Cherry pie", "Pork with beans"],
    "Amount": ["28 g", "100 g", "two", "237 ml", "170 g", "260 g"],
    "Calories": [110, 205, 160, 160, 420, 260],
    "Protein": [4, 32, 13, 8, 4, 14],
    "Calcium": [2, 12, 54, 285, 22, 80],
    "Price": [3.4, 24.01, 13.3, 9, 20, 19],
    "Max": [4, 4, 4, 4, 4, 4]
}).set_index("Food")

# Display the DataFrame

df

Note: you may need to restart the kernel to use updated packages.


,Amount,Calories,Protein,Calcium,Price,Max
Food,,,,,,
Oatmeals,28 g,110,4,2,3.40,4
Chicken,100 g,205,32,12,24.01,4
Eggs,two,160,13,54,13.30,4
Milk,237 ml,160,8,285,9.00,4
Cherry pie,170 g,420,4,22,20.00,4
Pork with beans,260 g,260,14,80,19.00,4


In [37]:
req = pd.Series([2500, 80, 8], index=["Calories", "Protein", "Calcium"])
req

Calories    2500
Protein       80
Calcium        8
dtype: int64

In [38]:
m = gp.Model("diet")
m.Params.LogToConsole = 0

# Create the variables
x = m.addVars(df.index, name="x")

# Set the objective
m.setObjective(x.prod(df.Price.to_dict()), GRB.MINIMIZE)

# Add constraints
m.addConstr(x.prod(df.Calories.to_dict()) >= req.Calories, "Calories")
m.addConstr(x.prod(df.Protein.to_dict()) >= req.Protein, "Protein")
m.addConstr(x.prod(df.Calcium.to_dict()) >= req.Calcium, "Calcium")

m.optimize()

# Die Lösung ausgeben
print(f"Gesamtkosten: {m.objVal}")
print("Die Lösung:")

for food in df.index:
    print(f"{food}: {x[food].x}")


Set parameter LogToConsole to value 0
Gesamtkosten: 77.27272727272727
Die Lösung:
Oatmeals: 22.727272727272727
Chicken: 0.0
Eggs: 0.0
Milk: 0.0
Cherry pie: 0.0
Pork with beans: 0.0


In [39]:
m.write("diet.lp")

with open("diet.lp") as f:
    print(f.read())

\ Model diet
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  3.4 x[Oatmeals] + 24.01 x[Chicken] + 13.3 x[Eggs] + 9 x[Milk]
   + 20 x[Cherry_pie] + 19 x[Pork_with_beans]
Subject To
 Calories: 110 x[Oatmeals] + 205 x[Chicken] + 160 x[Eggs] + 160 x[Milk]
   + 420 x[Cherry_pie] + 260 x[Pork_with_beans] >= 2500
 Protein: 4 x[Oatmeals] + 32 x[Chicken] + 13 x[Eggs] + 8 x[Milk]
   + 4 x[Cherry_pie] + 14 x[Pork_with_beans] >= 80
 Calcium: 2 x[Oatmeals] + 12 x[Chicken] + 54 x[Eggs] + 285 x[Milk]
   + 22 x[Cherry_pie] + 80 x[Pork_with_beans] >= 8
Bounds
End



In [40]:
m = gp.Model("diet-1")

# Entscheidungsvariablen

# x = m.addVars(..., ..., name="x")

# Zielfunktion

# m.setObjective(..., GRB.MINIMIZE)

# Bedingungen

# m.addConstr(..., "Calories")
# m.addConstr(..., "Protein")
# m.addConstr(..., "Calcium")

# m.optimize()

# DataFrame mit den Ergebnissen

# df["Optimal"] = [v.x for v in x.values()]
# df[["Optimal", "Price", "Calories"]]

## Neu Zealand Diet Problem

Der Datensatz `df` enthält die Kosten und die Nährstoffeigenschaften (ca. 140) von 883 Lebensmitteln in Neu Zealand (Quelle: <https://doi.org/10.1093/cdn/nzab132>)

- `ID`: Eindeutiger Identifikator des Lebensmittels
- `Name`: Name des Lebensmittels
- `Group`: Gruppe, zu der das Lebensmittel gehört
- `Cost`: Die Kosten per 100g des Lebensmittels
- `Amount`: Die maximal zu konsumierende Menge des Lebensmittels (in 100 g)
- `Dry matter`: Trockensubstanz des Lebensmittels (in g)
- `Energy, total metabolisable (kJ)`
- `Fat, total`: Gesamtfettgehalt des Lebensmittels (in g)
- `Protein, total; calculated from total nitrogen` (in g)
- `Total carbohydrates by summation` (in g)
- Sonstige Charakteristiken

Der Datensatz `reqs` enthält Ernährungsanforderungen für einen durchschnittlichen Erwachsenen.



In [41]:
# Einlesen der Lebensmittel-Daten

df = pd.read_csv('https://raw.githubusercontent.com/febse/data/refs/heads/main/opt/NZFoods.csv').set_index('ID')
df.head()

,Source,Group,SubGroup,Cost,Amount,Chapter,Name,Description,Alanine,Alpha-carotene,...,"Vitamin A, retinol equivalents",Vitamin B12,Vitamin B6,Vitamin C,Vitamin D; calculated by summation,"Vitamin E, alpha-tocopherol equivalents",Vitamin K,Water,Zeaxanthin,Zinc
ID,,,,,,,,,,,,,,,,,,,,,
M1170,Animal,Beef,Beef blade,2.042333,1.0,M,"Beef, forequarter bolar, separable lean & fat,...","Beef, forequarter bolar, separable lean & fat,...",1505.04,0.0,...,15.95,1.53,0.18,0.0,0.23,0.83,0.0,57.57,0.0,5.93
M1154,Animal,Beef,Beef blade,2.042333,1.0,M,"Beef, forequarter bolar, separable lean & fat,...","Beef, forequarter bolar, separable lean & fat,...",1036.61,0.0,...,8.88,1.71,0.32,0.0,0.14,0.45,0.0,70.26,0.0,3.46
M1069,Animal,Beef,Beef blade,2.042333,1.0,M,"Beef, forequarter bolar, separable lean, braised","Beef, forequarter bolar, separable lean, braised",1627.87,0.0,...,13.49,1.65,0.19,0.0,0.21,0.76,0.0,60.46,0.0,6.38
M1044,Animal,Beef,Beef blade,2.042333,1.0,M,"Beef, forequarter bolar, separable lean, raw","Beef, forequarter bolar, separable lean, raw",1072.58,0.0,...,6.25,1.80,0.33,0.0,0.13,0.40,0.0,73.11,0.0,3.63
M1171,Animal,Beef,Beef blade,2.042333,1.0,M,"Beef, forequarter brisket navel end, separable...","Beef, forequarter brisket navel end, separable...",870.80,0.0,...,27.91,0.84,0.09,0.0,0.31,1.11,0.0,42.28,0.0,3.70


In [42]:
# Einlesen der Ernährungsrichtlinien

reqs = pd.read_csv('https://raw.githubusercontent.com/febse/data/refs/heads/main/opt/NZPersonReq.csv')
reqs.head()

,Alanine,Alpha-carotene,Alpha-tocopherol,Arginine,Asparagine,Available carbohydrate by difference,Beta-carotene,Beta-carotene equivalents,Beta-tocopherol,Beta-tocopherol + Gamma-tocopherol,...,"Vitamin A, retinol equivalents",Vitamin B12,Vitamin B6,Vitamin C,Vitamin D; calculated by summation,"Vitamin E, alpha-tocopherol equivalents",Vitamin K,Water,Zeaxanthin,Zinc
0,0,0,0,0,0,0,0,0,0,0,...,800,2.400000e+00,1.3,45,5,8.5,65,0,0,11
1,99999999999,99999999999,99999999999,99999999999,99999999999,99999999999,99999999999,99999999999,99999999999,99999999999,...,3000,1.000000e+11,50.0,1000,80,300.0,99999999999,99999999999,99999999999,40


In [43]:
# Zugriff auf die Daten für ein bestimmtes Lebensmittel

df.loc["M1170"]

Source                                          Animal
Group                                             Beef
SubGroup                                   Beef blade 
Cost                                          2.042333
Amount                                             1.0
                                              ...     
Vitamin E, alpha-tocopherol equivalents           0.83
Vitamin K                                          0.0
Water                                            57.57
Zeaxanthin                                         0.0
Zinc                                              5.93
Name: M1170, Length: 145, dtype: object

In [44]:
m = gp.Model("Diet")
m.Params.LogToConsole = 0

# Entscheidungsvariablen (ID der Lebensmittel)

x = m.addVars(df.index, name="x")

# Zielfunktion

m.setObjective(x.prod(df['Cost'].to_dict()), GRB.MINIMIZE)

# Einschränkungen

for col in reqs.columns:
    m.addConstr(x.prod(df[col].to_dict()) >= reqs.loc[0, col], col)
    m.addConstr(x.prod(df[col].to_dict()) <= reqs.loc[1, col], col)

m.optimize()

print(f"Total Cost: {m.objVal}")

Set parameter LogToConsole to value 0
Total Cost: 2.804167442632024


In [45]:
# Hier speichern wir die Lösung in der Spalte "Optimal" des DataFrames mit den Lebensmittel-Daten

df["Optimal"] = [x[i].x for i in df.index]

# Das Menü anzeigen
# - Zuerst werden die Lebensmittel angezeigt, die in der optimalen Lösung enthalten sind
# - Dann werden nur die Spalten "Name", "Group", "Optimal", "Amount" und "Cost" angezeigt

df[df["Optimal"] > 0][["Name", "Group", "Optimal", "Amount", "Cost"]]

,Name,Group,Optimal,Amount,Cost
ID,,,,,
T1026,"Mussel, green, meat, boiled",Seafood,0.148860,1.0000,0.475000
F1026,"Milk, cow, standard 3.3% fat, fluid, Christchu...",Milk,3.500815,2.5750,0.173746
X23,"Cabbage, leafy vegetable, red",Vegetable,0.446254,0.6000,0.209083
X1039,"Cabbage, leafy vegetable, raw, tat soi",Vegetable,1.307076,0.6000,0.209083
X1137,"Potato, root vegetable, flesh & skin, baked ...",Vegetable,0.366369,1.3500,0.223250
X75,"Chickpea, seed vegetable, dried",Legumes,1.071478,1.3500,0.523571
A21,"Bread, white, toasted",Grain,0.346834,0.2600,0.218750
E17,"Flour, wheat, white, standard, upper North Island",Grain,4.672461,0.5000,0.135167
J1035,"Oil, soya bean",Fats and oils,0.137830,0.1380,0.360326


In [46]:
# Analyse der Lösung

df["EnergyOptimal"] = df["Energy, total metabolisable (kJ)"] * df["Optimal"]

# Die Fettmenge in der optimalen Lösung berechnen

df["FatOptimal"] = df["Fat, total"] * df["Optimal"]

# Die Proteinmenge in der optimalen Lösung berechnen

df["ProteinOptimal"] = df["Protein, total; calculated from total nitrogen"] * df["Optimal"]

# Die Kohlenhydratmenge in der optimalen Lösung berechnen

df["CarbohydrateOptimal"] = df["Total carbohydrates by summation"] * df["Optimal"]

df.aggregate({"EnergyOptimal": "sum", "FatOptimal": "sum", "ProteinOptimal": "sum", "CarbohydrateOptimal": "sum"})

EnergyOptimal          11150.000000
FatOptimal                58.491066
ProteinOptimal            92.515608
CarbohydrateOptimal      475.393903
dtype: float64

In [47]:
# Die Einschränkungen anzeigen (nur die bindenden Einschränkungen)

constr_df = pd.DataFrame(
    [(c.ConstrName, c.Sense, c.Slack, c.Pi, c.SARHSLow, c.SARHSUp) for c in m.getConstrs() if c.pi != 0],
    columns=["Name", "Sense", "Slack", "Shadow", "Lower", "Upper"]
    )
constr_df

,Name,Sense,Slack,Shadow,Lower,Upper
0,Biotin,>,0.0,0.006518,25.646329,61.741632
1,Calcium,>,0.0,0.000927,967.712384,1179.545899
2,Dietary folate equivalents,<,0.0,-0.000277,789.784567,1060.486688
3,"Energy, total metabolisable (kJ)",>,0.0,0.000006,10102.074880,11150.000000
4,Fatty acid 18:3 omega-3,>,0.0,0.026536,0.972971,1.362049
5,"Fatty acid cis,cis 18:2 omega-6",>,0.0,0.004134,7.506915,11.256698
6,Pantothenic acid,>,0.0,0.109966,4.125095,5.216972
7,Potassium,>,0.0,0.000258,3051.327614,3931.002695
8,Selenium,>,0.0,0.004293,63.649627,72.645262
9,Sodium,>,0.0,0.000071,380.892200,1611.752799


## Wie fukntionert der Code?

Wir haben im wesentlichen nur ein neues Element hinzugefügt und das ist die `for`-Schleife, die hier über die Spalten des DataFrames `dt` iteriert.

Um den Code zu verstehen, ist es hilfreich, die Dimension der Daten zu reduzieren, um den Datenfluß zu verstehen. Dazu können Sie die `print`-Anweisungen in den Schleifen einfügen.

In [78]:
# Zuerst erstellen wir ein Subset des DataFrames mit den Anforderungen, die nur die Spalten 
# "Alanine", "Alpha-carotene" und "Arginine" enthalten

reqs_subset = reqs[['Alanine', 'Alpha-carotene', 'Arginine']]
reqs_subset

,Alanine,Alpha-carotene,Arginine
0,0,0,0
1,99999999999,99999999999,99999999999


In [79]:
# Aus dem Datensatz mit the Lebensmittel-Daten erstellen wir auch ein Subset, das nur die Spalten
# "Cost", "Alanine", "Alpha-carotene" und "Arginine" enthält
# Weiterhin wählen wir nur die ersten drei Zeilen aus

df_subset = df[['Cost'] + list(reqs_subset.columns)][:3]
df_subset

,Cost,Alanine,Alpha-carotene,Arginine
ID,,,,
M1170,2.042333,1505.04,0.0,1902.60
M1154,2.042333,1036.61,0.0,1310.53
M1069,2.042333,1627.87,0.0,2057.87


In [48]:
# Im allgemeinen benutzen wir Schleifen, um eine oder mehrere Oprationen
# mit jedem Element einer Liste auszuführen.

# Beispiel: Berechnung der Summe der Zahlen von 1 bis 5

summe = 0

for i in [1, 2, 3, 4, 5]:        
    summe = summe + i
    print("i =", i)
    print("Summe =", summe)


i = 1
Summe = 1
i = 2
Summe = 3
i = 3
Summe = 6
i = 4
Summe = 10
i = 5
Summe = 15


In [80]:
reqs_subset.columns

Index(['Alanine', 'Alpha-carotene', 'Arginine'], dtype='object')

In [82]:
# Da der Datensatz mit den Lebensmitteln dieselben Spalten wie der Datensatz mit den Ernährungsrichtlinien hat,
# können wir diese Spalten folgendermassen abrufen

for col in reqs_subset.columns:
    print("Spalte:", col)
    # Mit loc greifen wir auf die Werte in der ersten Zeile (untere Grenze) 
    print("Untere Grenze:", reqs.loc[0, col])

    # Und hier tun wir das gleiche für die Werte in der zweiten Zeile (obere Grenze)
    print("Obere Grenze:", reqs.loc[1, col])
    

Spalte: Alanine
Untere Grenze: 0
Obere Grenze: 99999999999
Spalte: Alpha-carotene
Untere Grenze: 0
Obere Grenze: 99999999999
Spalte: Arginine
Untere Grenze: 0
Obere Grenze: 99999999999


In [85]:
for col in reqs_subset.columns:
    print("Spalte:", col)
    print("Untere Grenze:", reqs_subset.loc[0, col])
    print("Obere Grenze:", reqs_subset.loc[1, col])
    
    print("Mittelwert der Spalte mit demselben Namen in 'df_subset'", df_subset[col].mean())
    

Spalte: Alanine
Untere Grenze: 0
Obere Grenze: 99999999999
Mittelwert der Spalte mit demselben Namen in 'df_subset' 1389.84
Spalte: Alpha-carotene
Untere Grenze: 0
Obere Grenze: 99999999999
Mittelwert der Spalte mit demselben Namen in 'df_subset' 0.0
Spalte: Arginine
Untere Grenze: 0
Obere Grenze: 99999999999
Mittelwert der Spalte mit demselben Namen in 'df_subset' 1757.0


In [88]:
m2 = gp.Model("Diet-2")

x_subset = m2.addVars(df_subset.index, name="x")

m2.update()

for col in reqs_subset.columns:
    print("Spalte:", col)
    print("Untere Grenze:", reqs_subset.loc[0, col])
    print("Obere Grenze:", reqs_subset.loc[1, col])
    
    print(x_subset.prod(df_subset[col].to_dict()) >= reqs_subset.loc[0, col])
    print(x_subset.prod(df_subset[col].to_dict()) <= reqs_subset.loc[1, col])


Spalte: Alanine
Untere Grenze: 0
Obere Grenze: 99999999999
<gurobi.TempConstr: 1505.04 x[M1170] + 1036.61 x[M1154] + 1627.87 x[M1069] >= 0>
<gurobi.TempConstr: 1505.04 x[M1170] + 1036.61 x[M1154] + 1627.87 x[M1069] <= 99999999999>
Spalte: Alpha-carotene
Untere Grenze: 0
Obere Grenze: 99999999999
<gurobi.TempConstr: 0.0 x[M1170] + 0.0 x[M1154] + 0.0 x[M1069] >= 0>
<gurobi.TempConstr: 0.0 x[M1170] + 0.0 x[M1154] + 0.0 x[M1069] <= 99999999999>
Spalte: Arginine
Untere Grenze: 0
Obere Grenze: 99999999999
<gurobi.TempConstr: 1902.6 x[M1170] + 1310.53 x[M1154] + 2057.87 x[M1069] >= 0>
<gurobi.TempConstr: 1902.6 x[M1170] + 1310.53 x[M1154] + 2057.87 x[M1069] <= 99999999999>
